In [1]:
import os
import cv2
import numpy as np
import glob

In [6]:
if not os.path.isdir('images'): #이미지 디렉토리 없으면 만들어라
    os.mkdir('images') #프레임별 이미지 저장 / synth : 합성된 이미지

In [19]:
# src.mp4로부터 이미지 프레임 추출

src_idx = 0 #현재 프레임 저장

cap1 = cv2.VideoCapture('./data/src.mp4')
while True:
    retval, frame = cap1.read()
    if not retval:
        break
    file_name = f'./images/src/src_{src_idx:03d}.jpg' #프레임별로 번호 붙여서 저장
    cv2.imwrite(file_name, frame)
    src_idx += 1
    
if cap1.isOpened():
    cap1.release()

SyntaxError: invalid syntax (<ipython-input-19-910cb0403fd3>, line 11)

In [16]:
# dst.mp4로부터 이미지 프레임 추출
# src.mp4로부터 이미지 프레임 추출

dst_idx = 0 #현재 프레임 저장
cap2 = cv2.VideoCapture('./data/dst.mp4')
while True:
    retval, frame = cap2.read()
    if not retval:
        break
    file_name = f'./images/dst/dst_{dst_idx:03d}.jpg' #프레임별로 번호 붙여서 저장
    cv2.imwrite(file_name, frame)
    src_idx += 1
    
if cap1.isOpened():
    cap1.release()

SyntaxError: invalid syntax (<ipython-input-16-60e4db809469>, line 10)

In [20]:
#homography 계산 및 합성

pts_src = np.array([[0, 0], [1279, 0], [1279, 719], [0, 719]])
pts_dst = np.array([[232, 100], [606, 92], [602, 352], [236, 316]]) #점 찍음
h, status = cv2.findHomography(pts_src, pts_dst)

#이미지 합성 함수
def synthesize(im_src, im_dst):
    im_warp = cv2.warpPerspective(im_src, h, (im_dst.shape[1], im_dst.shape[0])) #적용해서 합성
    cv2.fillConvexPoly(im_dst, pts_dst.astype(int), (0, 0, 0), cv2.LINE_AA) #합성 / (0,0,0) : 합성 부위를 까맣게 칠함
    return im_warp + im_dst


In [21]:
#프레임별 이미지 합성
file_num = min(len(glob.glob('./images/src/src*.jpg')), len(glob.glob('images/dst/dst*.jpg')))

for i in range(file_num):
    im_src = cv2.imread('./images/src/src_' + str(i).zfill(3) + '.jpg')
    im_dest = cv2.imread('./images/dst/dst_' + str(i).zfill(3) + '.jpg')
    
    im_comp = synthesize(im_src, im_dest)
    
    cv2.imwrite('./images/synth/synth_' + str(i).zfill(3) + '.jpg', im_comp)

In [22]:
#최종 동영상 생성
file_list = os.listdir('./images/synth/')
fps = 20
fcc = cv2.VideoWrter_fourcc('D', 'I', 'V', 'X') #코덱 지정
width = 854
height = 480
out = cv2.VideoWriter('homography.avi', fcc, fps, (854, 480))

for f in file_list:
    fn = f'./images/synth/{f}'
    img = cv2.imread(fn)
    out.write(img)
    
out.release()

SyntaxError: invalid syntax (<ipython-input-22-a8a4cb6bfb9f>, line 10)